## 1. Configuração e Carregamento (Adaptado para CSV)

Importamos as bibliotecas e carregamos a base limpa. Atenção: usando CSV, precisamos especificar o separador ; e converter a coluna de data (DT_COMPTC) explicitamente.

In [6]:
import pandas as pd
import numpy as np
import gc

# Configuração de visualização (para ver mais colunas no print)
pd.set_option('display.max_columns', 50)

# Caminhos
PROCESSED_DIR = '../data/processed'
arquivo_entrada = f'{PROCESSED_DIR}/base_acoes_consolidada.csv'

print("📂 Carregando base de dados (CSV)...")

# Lendo o CSV (Dica: parse_dates ajuda a ler a data corretamente direto na leitura)
df = pd.read_csv(arquivo_entrada, sep=';', parse_dates=['DT_COMPTC'], low_memory=False)

# Garante a ordenação (Fundamental para cálculos de tempo)
df.sort_values(by=['CNPJ_FUNDO', 'DT_COMPTC'], inplace=True)

print(f"✅ Base carregada! Dimensões: {df.shape}")
print(f"Período: de {df['DT_COMPTC'].min()} até {df['DT_COMPTC'].max()}")
df.head()

📂 Carregando base de dados (CSV)...
✅ Base carregada! Dimensões: (5675, 7)
Período: de 2023-12-01 00:00:00 até 2025-12-17 00:00:00


,CNPJ_FUNDO,DT_COMPTC,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
0,06.985.322/0001-13,2023-12-01,9.583126,2.113408e+08,0.0,0.0,6
1,06.985.322/0001-13,2023-12-04,9.078131,2.002039e+08,0.0,0.0,6
2,06.985.322/0001-13,2023-12-05,9.077056,2.001802e+08,0.0,0.0,6
3,06.985.322/0001-13,2023-12-06,9.165784,2.021370e+08,0.0,0.0,6
4,06.985.322/0001-13,2023-12-07,9.262598,2.042721e+08,0.0,0.0,6


## 2. Criando Features de Retorno (O Passado)

Calculamos os retornos passados. O modelo precisa saber se o fundo vem performando bem ou mal para prever se vai entrar dinheiro.

- RET_DIARIO: Variação percentual de um dia para o outro.
- RET_ACC_X: Retorno acumulado em janelas móveis (21, 63, 126 dias).

In [7]:
print("🧮 Calculando Retornos...")

# Agrupa por fundo para não misturar dados de fundos diferentes
grouped = df.groupby('CNPJ_FUNDO')

# 1. Retorno Diário Simples: (Preço Hoje / Preço Ontem) - 1
df['RET_DIARIO'] = grouped['VL_QUOTA'].pct_change()

# 2. Retornos Acumulados (Janelas Móveis)
# Janelas: 21 dias (1 mês), 63 dias (1 trim), 126 dias (1 sem)
janelas = [21, 63, 126]

for w in janelas:
    col_name = f'RET_ACC_{w}D'
    print(f"  -> Calculando janela de {w} dias...")

    # pct_change(w) pega o valor de hoje e compara com w dias atrás
    df[col_name] = grouped['VL_QUOTA'].pct_change(periods=w)

# Removemos os dias iniciais onde não dá para calcular (ex: no dia 1 não tem retorno de 126 dias atrás)
# Opcional: Podemos manter NA ou preencher com 0. Vamos manter NA por enquanto e limpar depois.
print("\n🔍 Exemplo de um fundo aleatório:")
fundo_exemplo = df['CNPJ_FUNDO'].iloc[0]
cols_view = ['DT_COMPTC', 'VL_QUOTA', 'RET_DIARIO', 'RET_ACC_21D']
print(df[df['CNPJ_FUNDO'] == fundo_exemplo][cols_view].tail())

🧮 Calculando Retornos...
  -> Calculando janela de 21 dias...
  -> Calculando janela de 63 dias...
  -> Calculando janela de 126 dias...

🔍 Exemplo de um fundo aleatório:
     DT_COMPTC   VL_QUOTA  RET_DIARIO  RET_ACC_21D
511 2025-12-11  10.432598    0.005192     0.040648
512 2025-12-12  10.366041   -0.006380     0.050897
513 2025-12-15  10.373755    0.000744     0.032184
514 2025-12-16  10.728045    0.034152     0.065727
515 2025-12-17  10.788247    0.005612     0.078490


## 3. Risco e o Futuro (Target)

Agora calculamos a Volatilidade (risco) anualizada. Um fundo volátil assusta investidores conservadores.

E, o mais importante: criamos o TARGET.

- O Target é a soma da Captação Líquida dos próximos 21 dias.
- Normalizamos pelo PL (Patrimônio Líquido) para comparar fundos pequenos com gigantes.
- Truque: Invertemos a ordem dos dados para calcular a soma "para frente" (futuro) usando janelas móveis.

In [8]:
print("📉 Calculando Volatilidade (Risco)...")

# --- CORREÇÃO 1: Usando 'transform' para alinhar os índices automaticamente ---
# O transform garante que o resultado tenha o mesmo tamanho e ordem da tabela original
for w in janelas:
    col_vol = f'VOL_{w}D'
    # Calcula desvio padrão na janela, anualiza (raiz de 252) e salva
    df[col_vol] = grouped['RET_DIARIO'].transform(lambda x: x.rolling(window=w).std()) * np.sqrt(252)

print("🎯 Calculando o Target (O Futuro)...")

# 1. Calcular o Fluxo Diário
# Usamos transform + shift para pegar o PL de ontem de cada grupo corretamente
df['PL_LAG1'] = grouped['VL_PATRIM_LIQ'].transform(lambda x: x.shift(1))
df['FLOW_PCT_DIARIO'] = (df['CAPTC_DIA'] - df['RESG_DIA']) / df['PL_LAG1']

# --- CORREÇÃO 2: Estratégia Segura para o Futuro (Cópia Reversa) ---
# Em vez de tentar inverter e desinverter no mesmo lugar, criamos uma cópia auxiliar.
# Isso evita o erro de índice incompatível.

# A. Criamos uma cópia temporária invertida (do futuro para o passado)
df_reverso = df.iloc[::-1].copy()

# B. Calculamos a soma móvel nessa cópia (agrupando por fundo)
# Reset_index(0, drop=True) remove o índice do grupo para alinhar com o índice original
soma_futura = df_reverso.groupby('CNPJ_FUNDO')['FLOW_PCT_DIARIO'].rolling(window=21).sum().shift(1).reset_index(0, drop=True)

# C. Atribuímos o resultado à cópia reversa (O Pandas alinha pelo índice numérico automaticamente)
df_reverso['TARGET_TEMP'] = soma_futura

# D. Jogamos de volta para o DataFrame original (Alinhamento por índice garante a ordem certa)
df['TARGET_FLOW_21D'] = df_reverso['TARGET_TEMP']

print("🧹 Limpando dados e salvando...")

# Remove dados nulos gerados pelas janelas
df_modelagem = df.dropna(subset=['TARGET_FLOW_21D', 'RET_ACC_126D', 'VOL_126D']).copy()

# Selecionar colunas finais
cols_finais = [
    'CNPJ_FUNDO', 'DT_COMPTC',
    'TARGET_FLOW_21D',
    'RET_ACC_21D', 'RET_ACC_63D', 'RET_ACC_126D',
    'VOL_21D', 'VOL_63D', 'VOL_126D',
    'PL_LAG1'
]

df_modelagem = df_modelagem[cols_finais]

# Salva em CSV
arquivo_final = f'{PROCESSED_DIR}/base_modelagem.csv'
print(f"💾 Salvando base pronta: {arquivo_final}")
df_modelagem.to_csv(arquivo_final, index=False, sep=';')

print("-" * 40)
print(f"✅ SUCESSO! Base de Modelagem: {df_modelagem.shape} linhas.")
print(df_modelagem.tail())

📉 Calculando Volatilidade (Risco)...
🎯 Calculando o Target (O Futuro)...
🧹 Limpando dados e salvando...
💾 Salvando base pronta: ../data/processed/base_modelagem.csv
----------------------------------------
✅ SUCESSO! Base de Modelagem: (4058, 10) linhas.
              CNPJ_FUNDO  DT_COMPTC  TARGET_FLOW_21D  RET_ACC_21D  \
5649  46.729.394/0001-14 2025-11-11        -0.091332     0.039852   
5650  46.729.394/0001-14 2025-11-12        -0.091011     0.017414   
5651  46.729.394/0001-14 2025-11-13        -0.089594    -0.040204   
5652  46.729.394/0001-14 2025-11-14        -0.088690    -0.049848   
5653  46.729.394/0001-14 2025-11-17        -0.093540    -0.073430   

      RET_ACC_63D  RET_ACC_126D   VOL_21D   VOL_63D  VOL_126D     PL_LAG1  
5649     0.063013      0.104767  0.150579  0.100761  0.088394  3969858.97  
5650     0.041316      0.074202  0.171355  0.110438  0.093620  4098730.10  
5651    -0.013035      0.016031  0.247432  0.151442  0.118748  4005601.05  
5652    -0.009803      0.0